In [139]:
import numpy as np
from time import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%run Noise.ipynb
%run general_solver.ipynb

In [451]:
N = 2 # number of realizations of the noise and initial conditions

# time space boundaries
s, t, a, b = 0, 1, 0, 1

# time space mesh sizes
dt, dx = 1e-4, 1/2**7

# Generate N realizations of space time white noise. 
# Might take a while if the dt = 1e-4. Around 441 seconds to generate 1000 noises with mesh dt = 1e-4 on my laptop
# and 146 seconds with dt = 1e-3.
# Might be worth spliting into batches of 100 noises and apply general_1d_solver separately if dt = 1e-4. In general_1d_solver
# noise W is transformed into dW which might take a while if N is large and dt is small.

s_ = -time()
W = Noise().WN_space_time_many(s, t, dt, a, b, dx, N)
print(s_+time())

#space time domain
T, X = np.linspace(0,1, W.shape[1]), np.linspace(0,1, W.shape[2])

# generate N realizations of initial conditions like in the paper
u0 = np.array([[x*(1-x) for x in X[:-1]] for _ in range(N)])
lambd = 0.1 # strength of the randomness of the noise

u0 = u0 + lambd*Noise().initial(N, X, p = 10)[:,:-1] # Delete the last space point which is equal to the first one


0.23606300354003906


In [216]:
# Kuramoto–Sivashinsky SPDE.

L_ks = [0,0,-1,0,-1]
mu_ks = lambda x : 0
sigma_ks = lambda x : 1

KS, _, _ = general_1d_solver(L_ks, u0, W[:,:,:-1], mu = mu_ks, sigma = sigma_ks, KPZ = 0.5)

# Ignore the last space point in the noise which is equal to the first one by periodicity in W[:,:,:-1]

100%|██████████| 1000/1000 [00:00<00:00, 4919.42it/s]


In [18]:
# Random Nonlinear Schrodinger equation (no noise)

L_s = [0,0,1j,0,0]
mu_s = lambda x : -1j*x*np.absolute(x)**2
sigma_s = lambda x : 0

# create imaginary part of the initial condition

u0_im = u0 + lambd*Noise().initial(N, X, p = 10)[:,:-1]

u0_s = u0 + 1j*u0_im

Schrodinger, _, _ = general_1d_solver(L_s, u0_s, W[:,:,:-1], mu = mu_s, sigma = sigma_s, compl = True)

100%|██████████| 1000/1000 [00:00<00:00, 1060.31it/s]


In [36]:
# Phi^4 equation with Dirichlet boundary conditions

u0_d = np.array([[x*(1-x) for x in X] for _ in range(N)])
u0_d = u0_d + lambd*Noise().initial(N, X, p = 10, Dirichlet = True)

mu_d = lambda x: 3*x - x**3 
sigma_d = lambda x: 1

solver = SPDE(Type = 'P', IC = u0_d, mu = mu_d, sigma = sigma_d, BC = 'D', T = T, X = X)

Phi4_D = solver.Parabolic(W)

100%|██████████| 1000/1000 [00:00<00:00, 6379.01it/s]


In [427]:
# Cahn-Hilliard equation with Periodic boundary conditions

L_ch = [0,0,0,0,-1]
D_ch = [0,0,1,0,0]
mu_ch = lambda x: 3*x - x**3 
sigma_ch = lambda x: 1

Cahn_Hilliard, _, _ = general_1d_solver(L_ch, u0, W[:,:,:-1], mu = mu_ch, sigma = sigma_ch, D = D_ch)

100%|██████████| 10000/10000 [00:01<00:00, 9407.16it/s]


In [452]:
# smooth Q noise as in Example 10.8 of `An Introduction to Computational Stochastic PDEs' by Lord, Powell & Shardlow

def smooth_corr(x, j, a, r = 0):
    def q(j):
        if j == 0:
            return 0
        return (j//2+1)**(-r)
    
    return np.sqrt(q(j))*np.sqrt(2 / a) * np.sin(j * np.pi * x / a)


r = 4 # Creates r/2 spatially smooth noise
corr = lambda x, j, a : smooth_corr(x, j, a, r + 1.001)

W_smooth = Noise().WN_space_time_many(s, t, dt, a, b, dx, N, correlation = corr)

In [453]:
# KdV equation with Periodic boundary conditions

L_kdv = [0,0,0,-0.1,0]
mu_kdv = lambda x: 0 
sigma_kdv = lambda x: 1

KdV, _, _ = general_1d_solver(L_kdv, u0, W_smooth[:,:,:-1], mu = mu_kdv, sigma = sigma_kdv, Burgers= -6)

100%|██████████| 10000/10000 [00:02<00:00, 4954.83it/s]
